In [1]:
from gcsa.google_calendar import GoogleCalendar
import pathlib
import pandas as pd
from gcsa.serializers.event_serializer import EventSerializer
import pytz

In [2]:
cred = pathlib.Path('/Users/kdavis10/.config/.credentials')
cred_json = "pygooglecal.apps.googleusercontent.com.json"

try:
    gcal = GoogleCalendar(credentials_path=cred / cred_json)
except Exception as e:
    (cred / "token.pickle").unlink()
for i, item in enumerate(gcal.get_calendar_list()):
    print(i, item.id, item)

0 am9lof1vfbcr4rgcdqjtepdgqqgsta9d@import.calendar.google.com Canvas Calendars - (Keith Davis Calendar (Canvas))
1 nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.calendar.google.com None - (NYT Astronomy and Space Calendar)
2 nd.edu_ck4094isrp2khkbs07do4btuhs@group.calendar.google.com None - (ND-DVT Unavailability Calendar)
3 nd.edu_lq6iek13ph6gdplr3tin6moin4@group.calendar.google.com None - (High-Z Calendar)
4 nd.edu_fjc8me4hhtpi3rl2qbapk36av0@group.calendar.google.com None - (ND-DVT Calendar Archive)
5 nd.edu_31u09m2lj54rbh313672bkj4lg@group.calendar.google.com None - (ND-DVT Provisional Calendar)
6 nd.edu_ehcva1rnlqt22gd9fbfkgh03v4@group.calendar.google.com None - (ND-DVT Student Work Schedule)
7 c_ueposo14lq53cgtihncl329uvg@group.calendar.google.com None - (Todoist)
8 coscal@nd.edu None - (CoS College-wide Events & Conferences)
9 addressbook#contacts@group.v.calendar.google.com None - (Birthdays)
10 c_e097d400992a5bb01a265f4d4b1b9e8081f0660ac11dd1baaefd1aa8e28a046e@group.calendar.goo

In [3]:
SELECTED = ['NYT Astronomy and Space Calendar',
 'ND-DVT Provisional Calendar',
 'CoS College-wide Events & Conferences',
 'Birthdays',
 'http://universerevealed.nd.edu/?eme_ical=public',
 'Holidays in United States',
 'Axis Ludi Calendar',
 'Keith Davis',
 'Academic Calendar',
 'kdavis10@nd.edu',
 'ND-DVT Calendar']

In [79]:
cal_data = {}
for cal in gcal.get_calendar_list():
    if cal.summary in SELECTED:
        cal_data[cal.summary] = {}
        cal_data[cal.summary]["summary"] = cal.summary
        cal_data[cal.summary]["calid"] = cal.id
        cal_data[cal.summary]["description"] = cal.description
        cal_data[cal.summary]["bg_color"] = cal.background_color
        cal_data[cal.summary]["fg_color"] = cal.foreground_color
        cal_data[cal.summary]["events"] = gcal.get_events(calendar_id=cal.id)
cal_data

{'NYT Astronomy and Space Calendar': {'summary': 'NYT Astronomy and Space Calendar',
  'calid': 'nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.calendar.google.com',
  'description': "Never miss an eclipse, a meteor shower, a rocket launch or any other astronomical and space event that's out of this world.",
  'bg_color': '#9fc6e7',
  'fg_color': '#000000',
  'events': <generator object EventsService.get_events at 0x116715b90>},
 'ND-DVT Provisional Calendar': {'summary': 'ND-DVT Provisional Calendar',
  'calid': 'nd.edu_31u09m2lj54rbh313672bkj4lg@group.calendar.google.com',
  'description': 'This calendar is for unconfirmed DVT presentation requests.  Events will be placed on this calendar if they are too early to be confirmed, or if other requests conflict and we are awaiting resolution.',
  'bg_color': '#c2c2c2',
  'fg_color': '#000000',
  'events': <generator object EventsService.get_events at 0x116715cc0>},
 'CoS College-wide Events & Conferences': {'summary': 'CoS College-wide Even

In [80]:
def prep_event(event, cal_data):  
    sevent = EventSerializer.to_json(event)
    keep = ("id", "summary", "description", "htmlLink")
    clean = {}
    for k in keep:
        try:
            clean[k] = sevent[k]
        except KeyError as e:
            if k != "id":
                pass
            else:
                raise ValueError("Event does not have id. Something is wrong.")
               
    try:
        clean["start"] = pd.Timestamp(sevent["start"]["dateTime"], tz = sevent["start"]["timeZone"])
    except KeyError as e:
        clean["start"] = pd.Timestamp(sevent["start"]["date"])
    try:
        clean["end"] = pd.Timestamp(sevent["end"]["dateTime"], tz = sevent["end"]["timeZone"])
    except KeyError as e:
        clean["end"] = pd.Timestamp(sevent["end"]["date"])

    clean["color"] = cal_data["bg_color"]
    clean["calendar"] = cal_data["summary"]
    clean["calid"] = cal_data["calid"]
    
    return clean

In [81]:
grand_event_list = []
for cal in cal_data:
    print(cal)
    for event in cal_data[cal]["events"]:
        if event.start is not None:
            grand_event_list.append(prep_event(event, cal_data[cal]))
        else:
            print(f"Skipping {event}")

NYT Astronomy and Space Calendar
ND-DVT Provisional Calendar
CoS College-wide Events & Conferences
Birthdays
http://universerevealed.nd.edu/?eme_ical=public
Holidays in United States
Axis Ludi Calendar
Keith Davis
Academic Calendar
kdavis10@nd.edu
Skipping None - None
Skipping None - None
Skipping None - None
Skipping None - None
Skipping None - None
Skipping None - None
Skipping None - None
ND-DVT Calendar


In [89]:
df = pd.DataFrame(grand_event_list)
df["start"] = pd.to_datetime(df.start, utc=True).dt.tz_convert("America/Indiana/Indianapolis")
df["end"] = pd.to_datetime(df.end, utc=True).dt.tz_convert("America/Indiana/Indianapolis")

df.rename(columns=dict(summary="Label",
                       description="Summary", htmlLink="links",
                       calendar="tags",
                       id="internal ID",
                      start="Start Date",
                      end="End Date")).to_csv('text.csv', date_format="%Y-%m-%d %H:%M:%S")

In [87]:
df

,id,summary,description,htmlLink,start,end,color,calendar,calid
0,51e647khe62oredoekm9mc0lsu,The Orionid meteor shower will reach its peak.,<p>The Orionids are well-loved by meteor showe...,https://www.google.com/calendar/event?eid=NTFl...,2024-10-19 20:00:00-04:00,2024-10-21 20:00:00-04:00,#9fc6e7,NYT Astronomy and Space Calendar,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...
1,4qjcvtvo2mbetn4d8r8khvj0un,The Leonid meteor shower will reach its peak.,<p>The Leonids produce some of the fastest met...,https://www.google.com/calendar/event?eid=NHFq...,2024-11-15 19:00:00-05:00,2024-11-17 19:00:00-05:00,#9fc6e7,NYT Astronomy and Space Calendar,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...
2,6nc0e5ikgf9hpl44aagscbiscd,The Geminid meteor shower will reach its peak.,"<p>Caused by debris from an asteroid, the Gemi...",https://www.google.com/calendar/event?eid=Nm5j...,2024-12-12 19:00:00-05:00,2024-12-14 19:00:00-05:00,#9fc6e7,NYT Astronomy and Space Calendar,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...
3,5dppbfbibg1k4ai1nblkjdgg85,The Ursid meteor shower will reach its peak.,"<p>A winter solstice light show, meteors from ...",https://www.google.com/calendar/event?eid=NWRw...,2024-12-20 19:00:00-05:00,2024-12-22 19:00:00-05:00,#9fc6e7,NYT Astronomy and Space Calendar,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...
4,5q765i0pl980g4015at4imffui,The winter solstice.,<p>It’s the scientific start to winter in the ...,https://www.google.com/calendar/event?eid=NXE3...,2024-12-20 19:00:00-05:00,2024-12-21 19:00:00-05:00,#9fc6e7,NYT Astronomy and Space Calendar,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...
...,...,...,...,...,...,...,...,...,...
261,_8kojihhg8cp4cb9i74ojgb9k6kr48ba26d136b9k6cr42...,Terry Ehrman - Theology (Phil Presenting),"THEO 20888 Science, Theology, and Creation",https://www.google.com/calendar/event?eid=Xzhr...,2024-11-05 19:00:00-05:00,2024-11-05 20:00:00-05:00,#cd74e6,ND-DVT Calendar,nd.edu_58898lmg53s7v8vm4co8dmq8gk@group.calend...
262,_74pk6cpg711jgba16crjib9k892jeb9p611k2ba56op3e...,Science Communications Writing,JED 30142 Principles of Science Communication,https://www.google.com/calendar/event?eid=Xzc0...,2024-12-05 09:30:00-05:00,2024-12-05 10:45:00-05:00,#cd74e6,ND-DVT Calendar,nd.edu_58898lmg53s7v8vm4co8dmq8gk@group.calend...
263,_6kr3idhh65132b9n64q34b9k70q38b9o8l132b9o6cp42...,PLS Science 2,PLS 30412 Scientific Inquiry: Past & Present,https://www.google.com/calendar/event?eid=XzZr...,2024-12-06 09:30:00-05:00,2024-12-06 10:45:00-05:00,#cd74e6,ND-DVT Calendar,nd.edu_58898lmg53s7v8vm4co8dmq8gk@group.calend...
264,_612j4e1l64pjcba18op48b9k8kqj4ba288sjaba48ko32...,Cosmology 10240,Exoplanets and SETI\n \nPHYS 10240 Elementary ...,https://www.google.com/calendar/event?eid=XzYx...,2024-12-12 09:30:00-05:00,2024-12-12 10:45:00-05:00,#cd74e6,ND-DVT Calendar,nd.edu_58898lmg53s7v8vm4co8dmq8gk@group.calend...
